In [1]:
import os
# import Google Drive 套件
from google.colab import drive
# 將自己的雲端硬碟掛載上去
drive.mount('/content/gdrive')

os.chdir('./gdrive/MyDrive/Colab Notebooks/deep_learning/aoi')      # 檔案目錄

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
os.listdir('./../../../../../'), os.getcwd()

(['.config', 'gdrive', 'dataset', 'sample_data'],
 '/content/gdrive/MyDrive/Colab Notebooks/deep_learning/aoi')

In [3]:
import shutil
shutil.copyfile('./dataset.zip', './../../../../../dataset.zip')

'./../../../../../dataset.zip'

In [4]:
import zipfile
def zip_list(file_path):
    zf = zipfile.ZipFile(file_path, 'r')
    zf.extractall('./../../../../../')

file_path = './../../../../../dataset.zip'
zip_list(file_path)

In [5]:
os.remove('./../../../../../dataset.zip')

In [3]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import cv2
import numpy as np
import pandas as pd
import os
import forCNN as fcn
import functions as fc
import pickle
import random
import time
import importlib
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [4]:
!nvidia-smi

Tue May 25 14:07:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
random.randint(0, 100)

76

# 提取圖片位置

In [5]:
p = './../../../../../'
# p = './'
trainArray = pd.read_csv(p + 'dataset/train.csv').to_numpy()
trainDatas = p + 'dataset/train_images/' + trainArray[:, 0]
trainLabels = trainArray[:, 1]
trainLabels = trainLabels.astype(np.int32)

n = trainDatas.shape[0]
valn = int(n * 0.9)

valDatas = trainDatas[valn:]
trainDatas = trainDatas[:valn]

valLabels = torch.tensor(trainLabels[valn:], dtype=torch.long)
trainLabels = torch.tensor(trainLabels[:valn], dtype=torch.long)
trainDatas.shape, valDatas.shape, trainLabels.shape, valLabels.shape

((2275,), (253,), torch.Size([2275]), torch.Size([253]))

In [6]:
# 處理不平衡
k = [0] * 6
f = [[] for _ in range(6)]
for i in range(trainLabels.shape[0]):
    t = trainLabels[i].item()
    k[t] += 1
    f[t].append(trainDatas[i])
k

[618, 446, 87, 342, 214, 568]

In [7]:
m = 700
for i in range(6):
    c = len(f[i])
    c1 = m // c + 1
    f[i] = (f[i] * c1)[:m]

balTrainDatas = np.array(f)
balTrainLabels = torch.tensor([[i for i in range(6)]], dtype=torch.long).T.expand(6, m)
balTrainDatas.shape, balTrainLabels.shape

((6, 700), torch.Size([6, 700]))

# 建立 model, loss, opt 物件

In [ ]:
importlib.reload(fc)
importlib.reload(fcn)

<module 'forCNN' from '/content/gdrive/My Drive/Colab Notebooks/deep_learning/aoi/forCNN.py'>

In [8]:
# 各個 path
logPath = './logger/'
modulePath = './modules/res_module_self.pt'
accuPath = './modules/accu.pickle'

In [9]:
# 模型參數
lr = 0.0001
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

cuda:0


In [10]:
module = fcn.ResNet().to(device)
lossf = nn.CrossEntropyLoss().to(device)
opt = torch.optim.Adam(module.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)
modules = (module, lossf, opt)
len(list(module.parameters()))

72

In [15]:
logger = fc.create_logger(logPath, 'res_logger.txt')

In [ ]:
#  for i in logger.handlers:
#     logger.removeHandler(i)
# logger.handlers = []
# del(logger)

In [ ]:
# inputs = fc.pic2tensor(trainDatas[0:2]).to(device)
# labels = trainLabels[0:2].to(device)
# outputs = module(inputs)
# fc.getaccu(outputs.cpu(), labels.cpu())

# train1

In [11]:
pics = trainDatas[0:50]
inputs = fc.pic2tensor_2(pics).to(device)

In [12]:
outputs = module(inputs)

In [ ]:
accu = fc.getaccu(outputs.cpu(), labels.cpu())

In [11]:
# 各個 path
logPath = './logger/'
modulePath = './modules/res_module_self.pt'
accuPath = './modules/accu.pickle'

In [12]:
# 訓練參數
batch = 10     # 會自動乘以 6 最高到 600
epochs = 100

In [16]:
# load model
# module = fcn.ResNet().to(device)
# module.load_state_dict(torch.load(modulePath))

<All keys matched successfully>

In [13]:
!nvidia-smi

Tue May 25 14:07:28 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    76W / 149W |    513MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# del inputs
# torch.cuda.empty_cache()

In [ ]:
torch.save(module.state_dict(), modulePath)

In [16]:
# balTrainDatas.shape, balTrainLabels.shape
start = time.time()
logger.info('start at {}'.format(start))

for epoch in range(1, epochs+1):
    lossList = []
    trainIndex = 0
    trainLen = balTrainDatas.shape[1]
    while trainIndex < trainLen:
        pics = balTrainDatas[:, trainIndex:trainIndex + batch].reshape(-1)
        inputs = fc.pic2tensor_2(pics).to(device)
        labels = balTrainLabels[:, trainIndex:trainIndex + batch].reshape(-1).to(device)
        loss = fc.train(modules, inputs, labels, update=True)
        trainIndex += batch
        lossList.append(loss)
    sumLoss = sum(lossList) / len(lossList)

    # inputs = fc.pic2tensor_2(trainDatas).to(device)
    # labels = trainLabels.to(device)
    # trainoutputs = fcn.train(modules, inputs, trainLabels, update=False)
    # trainAccu = fc.getaccu(trainoutputs.cpu(), trainLabels.cpu())

    inputs = fc.pic2tensor_2(valDatas).to(device)
    labels = valLabels.to(device)
    valoutputs = fc.train(modules, inputs, labels, update=False)
    valAccu = fc.getaccu(valoutputs.cpu(), valLabels.cpu())

    train_time = time.time() - start

    logger.info('train {}/{}, loss: {}, val accu: {}, train time {}'.format(epoch, epochs, sumLoss, valAccu, train_time))
    # logger.info('train accu: {},  val accu: {}'.format(trainAccu, valAccu))

    torch.save(module.state_dict(), modulePath)
    

2021-05-25 14:07:45,113 INFO start at 1621951665.1135228
2021-05-25 14:09:48,648 INFO train 1/100, loss: 1.4509030887058803, val accu: 0.6284584980237155, train time 123.53501892089844
2021-05-25 14:11:52,216 INFO train 2/100, loss: 1.2407543403761727, val accu: 0.6877470355731226, train time 247.10274577140808


KeyboardInterrupt: ignored

# test 的部分

# 提取 test 資料

In [3]:
import torch
import torch.nn as nn
from torch import optim
import numpy as np
import cv2
import numpy as np
import pandas as pd
import os
import forCNN as fcn
import functions as fc
import pickle
import random
import time
import importlib

In [5]:
p = './../../../../../'
# p = './'
testArray = pd.read_csv(p + 'dataset/test.csv')
testDatas = p + 'dataset/test_images/' + testArray.to_numpy()[:, 0]
testDatas.shape

(10142,)

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
module = fcn.ResNet().to(device)

In [7]:
modulePath = './modules/res_module_self.pt'
module.load_state_dict(torch.load(modulePath))

<All keys matched successfully>

In [8]:
n = testDatas.shape[0]
index = 0
with torch.no_grad():
    outputs = []
    while index < n:
        inputs = fc.pic2tensor_2(testDatas[index:index+5]).to(device)
        out = module(inputs)
        out = torch.argmax(out, dim=1)
        outputs.append(out)
        index += 5
    outputs = torch.cat(outputs, dim=0)

In [9]:
outputs = outputs.cpu().clone().detach().numpy().astype(np.int32)

In [10]:
testArray['Label'] = outputs

In [13]:
testArray.to_csv('./pred/test.csv', index=False)

In [11]:
testArray

,ID,Label
0,test_00000.png,1
1,test_00001.png,2
2,test_00002.png,5
3,test_00003.png,0
4,test_00004.png,2
...,...,...
10137,test_10137.png,3
10138,test_10138.png,1
10139,test_10139.png,4
10140,test_10140.png,0


# 讀取 model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# device = torch.device("cpu")
# print(device)

In [ ]:
model = forCNN.CNN_model_ResNet_withlin().to(device)
model.load_state_dict(torch.load('./model/ResNet_withlin.pt'))

<All keys matched successfully>

In [ ]:
real_label = labeltensor(img_list)
real_label.shape

torch.Size([624])

In [ ]:
pred = []
for p in img_list:
    pic = path2pic([p])
    inputs = pic2tensor(pic).to(device)
    outputs = model(inputs)
    #print(outputs)
    label = torch.argmax(outputs, dim=1)
    pred.append(label.item())

/home/re6091020/.local/lib/python3.6/site-packages/torch/nn/modules/container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [ ]:
print(confusion_matrix(real_label, pred), f1_score(real_label, pred, average='macro'))

[[234   0]
 [389   1]] 0.2756030523416306


In [ ]:
c = 0
for i in range(pred.shape[0]):
    if pred[i] == real_label[i]:
        c += 1
print(c)

472


In [ ]:
472 / 624

0.7564102564102564

In [ ]:
real_label

tensor([1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
        0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,

In [ ]:
pred

tensor([1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
        1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
        1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,

# try

In [ ]:
img = img_list[0:10]

In [ ]:
pic = path2pic(img)
inputs = pic2tensor(pic).to(device)
label = labeltensor(img).to(device)
outputs = model(inputs)

In [ ]:
#a, b = model(inputs)

In [ ]:
#a.shape, b.shape

(torch.Size([10, 64, 64, 64]), torch.Size([10, 64, 64, 64]))

In [ ]:
#(a + b).shape

torch.Size([10, 64, 64, 64])

In [ ]:
loss = lossf(outputs, label)
opt.zero_grad()
loss.backward()
opt.step()

In [ ]:
list(model.parameters())[0].grad

tensor([[[[ 7.5836e-02,  6.9965e-02,  6.7458e-02,  ...,  6.0451e-02,
            6.5087e-02,  7.0379e-02],
          [ 4.8154e-02,  3.9363e-02,  3.2561e-02,  ...,  3.3706e-02,
            3.5649e-02,  3.7161e-02],
          [ 1.1053e-01,  1.0182e-01,  9.5142e-02,  ...,  9.1332e-02,
            9.2364e-02,  9.0924e-02],
          ...,
          [ 1.0330e-01,  9.8182e-02,  9.1860e-02,  ...,  9.0316e-02,
            8.9937e-02,  8.6556e-02],
          [ 5.2832e-02,  5.1646e-02,  4.5458e-02,  ...,  4.8980e-02,
            5.0016e-02,  4.6380e-02],
          [ 6.3065e-02,  6.0708e-02,  5.6852e-02,  ...,  6.0731e-02,
            6.2515e-02,  5.9568e-02]],

         [[ 1.2922e-01,  1.2221e-01,  1.1951e-01,  ...,  1.2026e-01,
            1.1215e-01,  1.1693e-01],
          [ 1.1653e-01,  1.0989e-01,  1.0924e-01,  ...,  1.0448e-01,
            9.6465e-02,  1.0549e-01],
          [ 1.8378e-01,  1.7492e-01,  1.7658e-01,  ...,  1.6435e-01,
            1.5362e-01,  1.6466e-01],
          ...,
     

In [ ]:
c = readpic(img[0])

In [ ]:
c.shape

(1183, 1592, 3)

In [ ]:
a = c.reshape(3, 1183, -1)

In [ ]:
a[2, :,:]

array([[199, 199, 198, ...,   0,   0,   0],
       [  0,   0,   0, ...,  77,  76,  76],
       [ 76,  76,  76, ..., 200, 200, 199],
       ...,
       [  0,   0,   0, ..., 203, 204, 204],
       [204, 199, 199, ..., 212, 212, 209],
       [209, 209, 208, ...,   0,   0,   0]], dtype=uint8)

In [ ]:
showpic(c)

In [ ]:
showpic(a)

error: OpenCV(4.4.0) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-h4wtvo23\opencv\modules\core\src\array.cpp:2492: error: (-206:Bad flag (parameter or structure field)) Unrecognized or unsupported array type in function 'cvGetMat'


In [ ]:
pic = path2pic(img)

In [ ]:
label = labeltensor(img)

In [ ]:
label

tensor([1, 1, 1, 1, 1, 0, 1, 1, 1, 1])

In [ ]:
inputs = pic2tensor(pic)

In [ ]:
inputs.shape

torch.Size([10, 3, 512, 512])

In [ ]:
output = model(inputs)

In [ ]:
output.shape

torch.Size([10, 2])

In [ ]:
output

tensor([[0.4986, 0.5014],
        [0.4986, 0.5014],
        [0.4986, 0.5014],
        [0.4985, 0.5015],
        [0.4986, 0.5014],
        [0.4986, 0.5014],
        [0.4986, 0.5014],
        [0.4986, 0.5014],
        [0.4986, 0.5014],
        [0.4986, 0.5014]], grad_fn=<SoftmaxBackward>)

In [ ]:
loss = lossf(output, label)

In [ ]:
loss

tensor(0.6932, grad_fn=<NllLossBackward>)

In [ ]:
m = nn.AdaptiveAvgPool2d((1, 2))

In [ ]:
m2 = nn.Softmax(dim=1)

In [ ]:
cc = torch.rand(5, 10, 5, 5)

In [ ]:
cc.shape

torch.Size([5, 10, 5, 5])

In [ ]:
m2(m(cc.sum(dim=1)).view(5, -1))

tensor([[0.5344, 0.4656],
        [0.4322, 0.5678],
        [0.4884, 0.5116],
        [0.4794, 0.5206],
        [0.5252, 0.4748]])

In [ ]:
opt.zero_grad()

In [ ]:
loss.backward()

In [ ]:
cc.sum(dim=1).shape

torch.Size([5, 5, 5])

In [ ]:
torch.cat([cc, cc], dim=1).shape

torch.Size([1, 10, 3, 3])

In [ ]:
cc.shape

torch.Size([1, 5, 3, 3])

In [ ]:
x = ['a', 'b', 'c', 'd']

In [ ]:
random.shuffle(x)

In [ ]:
x

['a', 'd', 'b', 'c']

In [ ]:
m = nn.BatchNorm2d(100, affine=False)

In [ ]:
input = torch.randn(20, 100, 35, 5)

In [ ]:
output = m(input)
output.shape

torch.Size([20, 100, 35, 5])

In [ ]:
input

tensor([[[[-9.7968e-01, -6.4188e-01,  4.6603e-01,  ..., -2.1631e+00,
           -4.3154e-01, -1.6649e+00],
          [ 1.4414e+00, -6.1513e-01, -3.1568e+00,  ..., -1.1350e-01,
           -8.5218e-01, -1.2249e+00],
          [ 2.2272e-01, -6.5440e-01, -3.3495e-01,  ..., -4.9887e-01,
            5.6789e-01,  5.3829e-01],
          ...,
          [-2.4939e-01, -1.0704e-01,  3.1782e-01,  ...,  7.6236e-01,
            8.9548e-01, -5.2407e-01],
          [ 1.7321e-01, -1.1205e-01,  3.8517e-01,  ..., -1.1900e+00,
           -1.8117e+00,  3.9378e-01],
          [-4.4211e-01,  1.0476e+00, -5.7073e-01,  ..., -5.5228e-01,
            1.0204e+00,  9.7347e-01]],

         [[-5.9344e-01, -4.8301e-01, -7.8285e-01,  ..., -7.8669e-02,
           -8.3180e-01,  1.3470e+00],
          [ 4.0594e-01,  1.9151e-01, -1.3906e+00,  ..., -7.9700e-01,
            4.0568e-01,  1.0527e+00],
          [ 7.1232e-01,  4.0175e-01,  1.1238e+00,  ...,  1.4880e+00,
           -1.8002e+00,  1.1879e+00],
          ...,
     